In [3]:
from ultralytics import solutions

solutions.ParkingPtsSelection()

requirements: Ultralytics requirement ['tkinter'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.


In [ ]:
import cv2
import json
from ultralytics import solutions

polygon_json_path = "./boxes_json/bounding_boxes.json"   # 車位邊框JSON
video_path = "../video/istockphoto_01.mp4"
model_path = "../models/v6.pt"
output_video_path = "parking_management.avi"

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# 初始化停車管理系統
management = solutions.ParkingManagement(model_path, margin=1)

# 從JSON文件中提取車位的邊界框數據
with open(polygon_json_path, 'r') as f:
    json_data = json.load(f)

while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break

    # 使用YOLOv8模型對當前幀進行車輛檢測和跟踪
    results = management.model.track(im0, persist=True, show=False)

    # 如果檢測到車輛，則提取檢測結果的邊界框和類別
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().tolist()    # 提取邊界框坐標
        clss = results[0].boxes.cls.cpu().tolist()  # 提取車輛類別
        management.process_data(json_data, im0, boxes, clss)    # 處理檢測結果並更新停車區域狀態

    management.display_frames(im0)  # 在當前幀上顯示檢測結果
    video_writer.write(im0) # 將當前幀寫入到輸出視頻文件

cap.release()
video_writer.release()
cv2.destroyAllWindows()